In [4]:
# pip install pandas lxml undetected-chromedriver selenium requests

# Librerias

In [5]:
import requests
import pandas as pd
import random

import os

from bs4 import BeautifulSoup as bs
import random
import time

import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import undetected_chromedriver as uc


from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel


In [6]:
# Ruta local al driver de Chrome
ruta_driver = r"C:\Users\diego\OneDrive\Escritorio\PyCharm\Chambas\chromedriver-win64/chromedriver.exe"

# Configuración de opciones del navegador
chrome_options = Options()
# chrome_options.add_argument('--headless')  # Ejecutar sin abrir ventana
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Obtención de IDS de chambas

In [7]:
service = Service(executable_path=ruta_driver)
browser = webdriver.Chrome(service=service, options=chrome_options)

In [10]:
# Abre Google
service = Service(executable_path=ruta_driver)
browser = webdriver.Chrome(service=service, options=chrome_options)

# Cambia el nombre según el puesto que busques.
# Esto tambien definira el nombre del Excel que se genere.
busqueda = 'especialista en implementacion de software'

busqueda = busqueda.replace(' ','-')

# Numero de pagina en el que inicia a buscar.
x=1
ids = []

while True:

    url = f'https://www.occ.com.mx/empleos/de-{busqueda}/?page={x}'
    browser.get(url)
    time.sleep(random.randint(10,12))
    html = browser.page_source
    soup = bs(html, 'lxml')
    articles = soup.find('div',{'class':'flex flex-col'}).find_all('div', class_='bg-bg-surface-default')
    x = x+1

    # Obtiene IDS de cada trabajo en la pagina
    for article in articles:
        id_jobs = article.get('id')
        ids.append(id_jobs)
        time.sleep(random.randint(1,3))
        print(id_jobs)

    ids = [jobs for jobs in ids if jobs is not None]

    if x > 1:  # Ajusta este número con base a las páginas que deseas buscar. Para mi 2 es suficiente, pero pudes poner las que gustes
        print("Deteniendo Busqueda.")
        break

browser.quit()
# Eliminar letras de las IDs para generar enlaces de busqueda más adelante.
ids = [id_muebles.split('-')[-1] for id_muebles in ids]
print(ids)

jobcard-20605828
jobcard-20612197
jobcard-20665279
jobcard-20652795
jobcard-20647492
jobcard-20568473
jobcard-20645495
jobcard-20575437
jobcard-20642450
Deteniendo Busqueda.
['20605828', '20612197', '20665279', '20652795', '20647492', '20568473', '20645495', '20575437', '20642450']


In [11]:
# Elegir 15 IDs Aletoriamente
ids_filtro = random.sample(ids, 2)

# Formación y guardado de Enlaces de Chambas
link_chambas = [f'https://www.occ.com.mx/empleos/de-{busqueda}/?jobid={id}' for id in ids_filtro]
link_chambas

['https://www.occ.com.mx/empleos/de-especialista-en-implementacion-de-software/?jobid=20612197',
 'https://www.occ.com.mx/empleos/de-especialista-en-implementacion-de-software/?jobid=20575437']

# Scrapping de Trabajos y Exportación EXCEL

In [16]:
service = Service(executable_path=ruta_driver)
browser = webdriver.Chrome(service=service, options=chrome_options)
browser.get(url)

In [14]:
soup = bs(browser.page_source, 'html.parser')
soup

<html class="windows desktop portrait" lang="es-MX"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link as="font" crossorigin="" href="/serp/_next/static/media/7b28b12c04cb16fe-s.p.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="" href="/serp/_next/static/media/7e50924c6d8cd8b4-s.p.woff2" rel="preload" type="font/woff2"/><link as="image" fetchpriority="high" href="https://cdn-h4.occ.com.mx/images/logos/300x300/58/584e00ef57574fa59086485d8626a09a.jpg" rel="preload"/><link data-precedence="next" href="/serp/_next/static/css/a793b3b1d81675d0.css" rel="stylesheet"/><link as="script" fetchpriority="low" href="/serp/_next/static/chunks/webpack-d32046228cf0b6bb.js" rel="preload"/><script async="" src="https://tag.aticdn.net/piano-analytics.js" type="text/javascript"></script><script async="" src="https://d2r1yp2w7bby2u.cloudfront.net/js/clevertap.min.js" type="text/javascript"></script><script async="" src="https://www.g

In [2]:
try:
    browser.find_element(By.CLASS_NAME, "font-h5 text-white").click()
    print("Click realizado en el header")
except Exception as e:
    print("Nada")
    print(e)
    pass

Nada
name 'browser' is not defined


In [23]:
data = []

for url in link_chambas:

    service = Service(executable_path=ruta_driver)
    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.get(url)

    # Tiempo de Espera
    # Te recomiendo no moverle a este, se que es tardado, pero creeme.
    # No quieres ver como el scrapper deja de funcionar porque OCC te pide responder un captcha.
    wait_time = random.uniform(3, 10)
    print(f"Esperando {wait_time:.2f} segundos antes de continuar...")
    time.sleep(wait_time)

    # Obtiene el contenido HTML de la página
    soup = bs(browser.page_source, 'html.parser')
    try:
        browser.find_elements(By.CLASS_NAME, "font-h5 text-white")
        print("Click realizado")
    except:
        print("Nada")
    pass

    # Obtiene el contenido HTML de la página
    soup = bs(browser.page_source, 'html.parser')

    vacante = {}

    # TITULO
    try:
        vacante["Título"] = soup.find('p', {'class': 'text-[24px] leading-[110%]'}).text
        #print(f"Título obtenido: {titulo}")
    except AttributeError:
        print(f"No se encontró el título en: {url}")

    # UBICACIÓN
    try:
        vacante["Ubicación"] = soup.find('label',{'class':'font-light ml-1'}).text.split(',')[0]
        print(f"Ubicación obtenida:", vacante["Ubicación"])
    except AttributeError:
        print(f"No se encontró el lugar en: {url}")

    # Paga
    try:
        div_principal = soup.find('div', {'class': 'flex flex-wrap gap-x-1 mt-1 items-center'})

        vacante["Paga"] = div_principal.find('p', class_= 'text-[18px] leading-6').text.strip()

        print(f"Paga obtenida:", vacante["Paga"])
    except AttributeError:
        print(f"No se encontró la Paga en el formato especificado.")


    # EMPRESA
    try:
        vacante["Empresa"] = soup.select_one('div[class="mt-1"] span.line-clamp-1').text.strip()
        print(f"Empresa obtenida:", vacante["Empresa"])
    except AttributeError:
        print(f"No se encontró la empresa en: {url}")

    # Las SECCIÓNES "Sobre la empresa" y "Detalles" suelen ser relleno, la que realmente contiene la información importante es "DESCRIPCIÓN"
    # INFO (Sobre el Empleo)
    try:
        info = soup.find('div',{'class':'flex flex-col gap-2'})
        vacante["I_About"] = info.get_text(separator='\n').strip()
        #print(f"Info obtenida: {informacion_completa}")
    except AttributeError:
        print(f"No se encontró la info en: {url}")

    # INFO (Detalles)
    try:
        info = soup.find('div',{'class':'[&>div]:flex [&>div]:mb-1'})
        vacante["I_Detalles"] = info.get_text(separator='\n').strip()
        #print(f"Info obtenida: {informacion_completa}")
    except AttributeError:
        print(f"No se encontró la info en: {url}")

    # INFO (Descripción)
    try:
        info = soup.find('div',{'class':'break-words mb-8'})
        vacante["I_Descripción"] = info.get_text(separator='\n').strip()
        #print(f"Info obtenida: {informacion_completa}")
    except AttributeError:
        print(f"No se encontró la info en: {url}")
    data.append(vacante)

    # ENLACE
    try:
        vacante["Enlace"]=url
    except AttributeError:
        print("No se pudo guardar el enlace")
    print("")

    browser.quit()

# Crear DataFrame con los Datos
df = pd.DataFrame(data)

# Exportar datos a Excel
nombre_archivo = f"OCC_{busqueda.replace(' ', '_')}.xlsx"
df.to_excel(nombre_archivo, index=False, engine="openpyxl")

print("Los datos se han exportado a vacantes.xlsx correctamente.")


Esperando 9.19 segundos antes de continuar...
Click realizado
No se encontró el título en: https://www.occ.com.mx/empleos/de-especialista-en-implementacion-de-software/?jobid=20642450
No se encontró el lugar en: https://www.occ.com.mx/empleos/de-especialista-en-implementacion-de-software/?jobid=20642450
No se encontró la Paga en el formato especificado.
No se encontró la empresa en: https://www.occ.com.mx/empleos/de-especialista-en-implementacion-de-software/?jobid=20642450
No se encontró la info en: https://www.occ.com.mx/empleos/de-especialista-en-implementacion-de-software/?jobid=20642450
No se encontró la info en: https://www.occ.com.mx/empleos/de-especialista-en-implementacion-de-software/?jobid=20642450

Esperando 7.18 segundos antes de continuar...
Click realizado
Ubicación obtenida: Ciudad de México
Paga obtenida: Si el reclutador te contacta podrás conocer el sueldo
Empresa obtenida: Weldmation de México, S.A. de C.V.  en
Ciudad de México

Los datos se han exportado a vacantes